# <font color='yellow' style='font-size: 30px;'>1. Import dependencies</font>

In [ ]:
%pip install stable-baselines3[extra]

In [ ]:
import os
import gym
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from gym.utils.env_checker import check_env



# <font color='yellow' style='font-size: 30px;'>2. Load Environment</font> 

In [ ]:
env_name = "CartPole-v0"
env = gym.make(env_name)
env.reset()


In [ ]:
check_env(env)

In [ ]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render(mode='rgb_array')
        action = env.action_space.sample()
        n_stable, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))


In [ ]:
env.close()

## <font color="#888">Understanding The Environment</font>

In [ ]:
env.action_space

In [ ]:
env.action_space.sample()

In [ ]:
env.observation_space

In [ ]:
env.observation_space.sample()

# 3. Train an RL Model

In [ ]:
log_path = os.path.join('Training', 'logs')
log_path

In [ ]:
env = gym.make('CartPole-v0')
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
PPO??

In [ ]:
model.learn(total_timesteps=20000)


# 4. Save and Reload Model

In [ ]:
ppo_path = os.path.join('Training', 'Saved Models', 'ppo-models-cartpole')

In [ ]:
model.save(ppo_path)

In [ ]:
del model

In [ ]:
model = PPO.load(ppo_path, env=env)

# 5. Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

# 6. Test Model

In [ ]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render(mode='rgb_array')
        action, _ = model.predict(obs) # now using model here
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))

In [ ]:
env.close()

In [ ]:
obs = env.reset()

In [ ]:
model.predict(obs)

In [ ]:
env.action_space.sample()

# 7. Viewing Logs in Tensorboard

In [ ]:
training_log_path = os.path.join(log_path, 'ppo_2')

In [5]:
!tensorboard --logdir={training_log_path}

^C


# 8. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold


In [ ]:
save_path = os.path.join('Training', 'Saved Models')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)


In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=2000, callback=eval_callback)


# 9. Changing Policies

In [1]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
net_arch

[{'pi': [128, 128, 128, 128], 'vf': [128, 128, 128, 128]}]

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=2000, callback=eval_callback)


# 10. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=2000, callback=eval_callback)